# Linear Programming

This notebook:

- an introduction to linear programming,
- food/diet example (show),
- example of dispatching an energy system with scenario analysis (do together),
- port/market exercise (do alone).

Make sure to at least [glance through the notes](https://github.com/ADGEfficiency/teaching-monolith/blob/master/linear-programming/linear-programming.ipynb)!


## Linear programming with PuLP

We can do linear programming in Python using [PuLP](https://pythonhosted.org/PuLP/) - a library that abstracts away the mechanics of solving linear programs.  

There are many other solvers - most Python LP libraries will wrap around a lower level implementation (I hope!).


The mathematics of the optimization is not covered here - if you are interested, the [Simplex Method](https://en.wikipedia.org/wiki/Simplex_algorithm) is a good place to start.


## Example - Diet Problem

From [Linear programming - Michel Goemans](https://math.mit.edu/~goemans/18310S15/lpnotes310.pdf).

> In the diet model, a list of available foods is given together with the nutrient content and the cost
per unit weight of each food. A certain amount of each nutrient is required per day. For example,
here is the data corresponding to a civilization with just two types of grains (G1 and G2) and three
types of nutrients (starch, proteins, vitamins)

| | Starch [kg/kg] | Protiens [kg/kg] | Vitamins [kg/kg] | Cost [$/g] |
|---|---|---|---|---|
|apples| 5 | 4 | 2| 0.6 |
|oranges| 7 | 2 | 1| 0.35 |

We have a requirement per day of 8g of starch, 15g of protiens and 3g of vitamins.

### Formulating as a linear problem

Let's map this problem directly onto the definition of linear programming

1. Objective function = minimize cost,
2. Variables = amount of apples & oranges,
3. Constraints = daily requirements of starch, protien and vitamins.


## The PuLP api

Below is a full linear program for the problem above:

In [19]:
!pip install pulp numpy -Uq

from pulp import LpProblem, LpMinimize, LpVariable, LpStatus

problem = LpProblem('diet cost minimization', LpMinimize)

apples = LpVariable('apples', cat='Integer')
oranges = LpVariable('oranges', cat='Integer')

#  add the objective function
problem += apples * 0.6 + oranges * 0.35

#  add constraints
#  starch
problem += apples * 5 + oranges * 7 >= 8
#  protien
problem += apples * 4 + oranges * 2 >= 15
#  vitamins
problem += apples * 2 + oranges * 1 >= 3

#  run problem & show results
problem.solve()
print(LpStatus[problem.status])
for v in (apples, oranges):
    print('{} {}'.format(v.name, v.varValue))

You should consider upgrading via the '/Users/adam/.pyenv/versions/3.8.5/envs/general/bin/python3.8 -m pip install --upgrade pip' command.
Optimal
apples 5.0
oranges -2.0


What has happened here?  Our program has done something quite wierd!

Let's try again, putting some more constraints on our variables.

You could also achieve this by an explicit constraint (something like `prob += apples >=0` for a minimum of zero).  
`puLp` lets us do this when we create the variables.

In [6]:
prob = LpProblem('diet cost minimization', LpMinimize)
apples = LpVariable('apples', cat='Integer', lowBound=0, upBound=None)
oranges = LpVariable('oranges', cat='Integer', lowBound=0, upBound=None)

#  add the objective function
prob += apples * 0.6 + oranges * 0.35

#  add constraints
prob += apples * 5 + oranges * 7 >= 8
prob += apples * 4 + oranges * 2 >= 15
prob += apples * 2 + oranges * 1 >= 3

prob.solve()
print(LpStatus[prob.status])
for v in (apples, oranges):
    print('{} {}'.format(v.name, v.varValue))

Optimal
apples 4.0
oranges 0.0


Now we have a model, we can play with it.  

## Example - electricity system dispatch

Three assets in our grid:

- wind turbine,
- gas turbine,
- big bad coal.

Lets make 

In [2]:
from dataclasses import dataclass

@dataclass
class Asset:
    name: str
    price: float
    carbon_intensity: float
    limit: int

assets = [
    #  name, price $/MWh, carbon intensity tC/MWh
    ('wind', 30, 0.05, 100),
    ('gas', 70, 0.1, 50),
    ('coal', 50, 0.1, 100),
]
assets = [Asset(*a) for a in assets]

problem = LpProblem('cost minimization', LpMinimize)
#  make one variable per asset
variables = [LpVariable(a.name, 0, a.limit) for a in assets]

Let's create an objective function based on price:

In [3]:
problem += sum([a.price * v for a, v in zip(assets, variables)])

/Users/adam/.pyenv/versions/3.8.5/envs/general/lib/python3.8/site-packages/pulp/pulp.py:1671: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


And place one constraint - that the sum of our generation is equal to demand:

In [4]:
demand = 10
problem += sum(variables) == demand

problem.solve()
print(LpStatus[problem.status])

for v in variables:
    print('{} {}'.format(v.name, v.varValue))

Optimal
wind 10.0
gas 0.0
coal 0.0


Now we have a model - we can use it for scenario analysis:

In [10]:
for demand in [10, 50, 100]:
    
    problem = LpProblem('cost minimization', LpMinimize)
    assets = [
        #  name, price $/MWh, carbon intensity tC/MWh
        ('wind', 30, 0.05, 25),
        ('gas', 70, 0.1, 50),
        ('coal', 50, 0.1, 100),
    ]
    assets = [Asset(*a) for a in assets]
    variables = [LpVariable(a.name, 0, a.limit) for a in assets]
    
    problem += sum([a.price * v for a, v in zip(assets, variables)])
    problem += sum(variables) == demand
    problem.solve()
    print(LpStatus[problem.status])

    for v in variables:
        print('{} {}'.format(v.name, v.varValue))
    print('')

Optimal
wind 10.0
gas 0.0
coal 0.0

Optimal
wind 25.0
gas 0.0
coal 25.0

Optimal
wind 25.0
gas 0.0
coal 75.0



We can also look at what changing prices does to our dispatch:

## Transportation problem

$P$ ports
- each port has a capacity, measured in number of units

$M$ markets
- each market has a demand, measured in number of units

Each $(P,M)$ pair has a cost - the cost to transport goods from port $P$ to market $M$.

We want to find the lowest cost way to supply all our market demands from our ports.

We can map this problem directly onto the definition of linear programming:

- objective function = ?
- variables = ?
- constraints = ?

## Exercise

Build a linear program to solve the transportation problem:

In [10]:
ports = [20, 30, 30, 50]
markets = [20, 10, 5]

#  one price for a port-market pair - just randomly make prices
np.random.seed(42)
pm_cost = np.random.uniform(0, 1, size=len(ports) * len(markets)).reshape(len(ports), len(markets))

We can access the cost to trade from a port to a market by indexing `pm_cost[port, market]`:

In [16]:
#  trade from port 0 to market 0
pm_cost[0, 0]

0.3745401188473625

In [17]:
#  trade from port 3 to market 2
pm_cost[3, 2]

0.9699098521619943

Feel free to look at the answers by commenting out the two lines below:

In [15]:
#from answers import transportation
#transportation??